### 메모리 지우는 방법

In [282]:
# import gc
# del 변수이름
# gc.collect()

In [283]:
! python --version

Python 3.7.2


In [9]:
from azure.storage.blob import BlobServiceClient
import json
import os, sys
import pandas as pd
import numpy as np

In [10]:
account_name = 'ulsanbuk01'
conn_str = 'DefaultEndpointsProtocol=https;AccountName=seodaemun01;AccountKey=LpAXUpqtWPzGqBq+NS7rgTNwK2SCXSya1fBiXigsywsYmX7s8w5ZKF/dcsU1p9ZDQTpZ4LbNWzq8A1VKStM+1Q==;EndpointSuffix=core.windows.net'
container = 'dmsbi'
service = 'vms'
target = 'vms_status'
target_blob = 'vms/2021/08/26/05' # 실제 파일 크기 1.36MB

In [11]:
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

In [12]:
container_client = blob_service_client.get_container_client(container)

In [13]:
blob_client = container_client.get_blob_client(target_blob)

In [14]:
blob_client.exists()

False

In [15]:
stream = blob_client.download_blob()

ResourceNotFoundError: The specified blob does not exist.
RequestId:e1de8aba-201e-004d-14bb-9da746000000
Time:2021-08-30T16:24:25.2005320Z
ErrorCode:BlobNotFound
Error:None

In [291]:
data = stream.readall()

In [292]:
# data

In [293]:
print('데이터 타입 :', type(data))
print('데이터 길이 :', len(data))
print('데이터 메모리 용량 : {0}MB'.format(round(sys.getsizeof(data)/1024/1024, 2) ))

데이터 타입 : <class 'bytes'>
데이터 길이 : 75368158
데이터 메모리 용량 : 71.88MB


In [294]:
data_str = data.decode('utf-8')
print('데이터 타입 :', type(data_str))
print('데이터 길이 :', len(data_str))
print('데이터 메모리 용량 : {0}MB'.format(round(sys.getsizeof(data_str)/1024/1024, 2) ))

데이터 타입 : <class 'str'>
데이터 길이 : 75368158
데이터 메모리 용량 : 71.88MB


In [295]:
def getSize(array):
    return sys.getsizeof(array) + len(array) * sys.getsizeof(array[0])

data_pro = data_str.split('\n')
print('데이터 타입 :', type(data_pro))
print('데이터 길이 :', len(data_pro))
print('데이터 메모리 용량 : {0}MB'.format(round(getSize(data_pro)/1024/1024, 2) ))

데이터 타입 : <class 'list'>
데이터 길이 : 12279
데이터 메모리 용량 : 29.93MB


- 데이터 타입이 바이트인 상태에서 모두 합친 후 문자열 변환 및 SPLIT 하여 json_normalize 실행
- 각 소스 실행 후 gc.collect() 처리하여 메모리 사용량 비교할 것

In [296]:
res = data + data
res_ls = res.decode('utf-8').split('\n')[:-1]
res_json_ls = [ json.loads(item) for item in res_ls ]
df = pd.json_normalize(res_json_ls)
df

,collectTimeUTC,site,targetName,targetClass,dataType,yearPath,monthPath,dayPath,hourPath,data.cmd,data.cnt,data.msg,data.param.servers,data.res,data.time
0,2021-08-26 05:04:08.000,seodaemun,innodep,vms,history,2021,08,26,05,26,-1,0,"[{'app_type': 0, 'groups': [{'group_id': 1, 'i...",None,132744276051076930
1,2021-08-26 05:04:18.000,seodaemun,innodep,vms,history,2021,08,26,05,26,-1,0,"[{'app_type': 0, 'groups': [{'group_id': 1, 'i...",None,132744276150915000
2,2021-08-26 05:04:28.000,seodaemun,innodep,vms,history,2021,08,26,05,26,-1,0,"[{'app_type': 0, 'groups': [{'group_id': 1, 'i...",None,132744276251148060
3,2021-08-26 05:04:38.000,seodaemun,innodep,vms,history,2021,08,26,05,26,-1,0,"[{'app_type': 0, 'groups': [{'group_id': 1, 'i...",None,132744276350965700
4,2021-08-26 05:04:48.000,seodaemun,innodep,vms,history,2021,08,26,05,26,-1,0,"[{'app_type': 0, 'groups': [{'group_id': 1, 'i...",None,132744276451021540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24551,2021-08-26 06:58:16.000,seodaemun,innodep,vms,history,2021,08,26,05,26,-1,0,"[{'app_type': 1, 'groups': [{'group_id': 1, 'i...",None,132744344533139300
24552,2021-08-26 06:58:16.000,seodaemun,innodep,vms,history,2021,08,26,05,26,-1,0,"[{'app_type': 1, 'groups': [{'group_id': 1, 'i...",None,132744344533139300
24553,2021-08-26 06:58:17.000,seodaemun,innodep,vms,history,2021,08,26,05,26,-1,0,"[{'app_type': 1, 'groups': [{'group_id': 1, 'i...",None,132744344533139300
24554,2021-08-26 07:48:44.000,seodaemun,innodep,vms,history,2021,08,26,05,26,-1,0,"[{'app_type': 1, 'groups': [{'group_id': 2, 'i...",None,132744374801800050


In [297]:
df['data.param.servers'].explode('data.param.servers')[0].keys()

dict_keys(['app_type', 'groups', 'srv_addr', 'srv_name', 'srv_serial', 'status', 'type', 'vms_id'])

In [298]:
df_ = pd.json_normalize(df['data.param.servers'].explode('data.param.servers'), record_path = 'groups', meta=['srv_name', 'srv_serial']).explode('items').reset_index().drop(['index'], axis=1)
df_items = pd.json_normalize(df_['items'])
df_cc = pd.concat([df_, df_items], axis=1).drop('items', axis=1)
df_cc.loc[:,'item_time'] = pd.to_datetime(df_cc['item_time']/10000000 - 11644473600, unit='s').astype('datetime64[s]')
df_cc.loc[:,'item_param'] = df_cc['item_param'].apply(lambda x : 'null' if 'null' in x else list(json.loads(x).values())[0])
df_cc.loc[:,'item_value_str'] = df_cc['item_value'].apply(lambda x : np.nan if x.replace('.','').isdecimal() and x.count('.') <= 1 else x)
df_cc.loc[:,'item_value']= df_cc['item_value'].apply(lambda x : pd.to_numeric(x) if x.replace('.','').isdecimal() and x.count('.') <= 1 else np.nan)
df_cc = df_cc.rename(columns={'group_id':'groups_id'})

In [299]:
df_

,group_id,items,status,srv_name,srv_serial
0,1,"{'item_changed': False, 'item_id': 1, 'item_pa...",normal,VURIXMS,11
1,1,"{'item_changed': False, 'item_id': 2, 'item_pa...",normal,VURIXMS,11
2,1,"{'item_changed': False, 'item_id': 3, 'item_pa...",normal,VURIXMS,11
3,1,"{'item_changed': False, 'item_id': 4, 'item_pa...",normal,VURIXMS,11
4,3,"{'item_changed': False, 'item_id': 1, 'item_pa...",normal,VURIXMS,11
...,...,...,...,...,...
849277,2,"{'item_changed': False, 'item_id': 2, 'item_pa...",normal,VURIX02,102
849278,2,"{'item_changed': False, 'item_id': 2, 'item_pa...",normal,VURIX02,102
849279,2,"{'item_changed': False, 'item_id': 2, 'item_pa...",normal,VURIX02,102
849280,2,"{'item_changed': False, 'item_id': 2, 'item_pa...",normal,VURIX02,102


In [300]:
df_cc

,groups_id,status,srv_name,srv_serial,item_changed,item_id,item_param,item_save,item_time,item_value,status,item_value_str
0,1,normal,VURIXMS,11,False,1,null,True,2021-08-26 05:00:00,5.0,normal,NaN
1,1,normal,VURIXMS,11,False,2,null,True,2021-08-26 05:00:00,32.0,normal,NaN
2,1,normal,VURIXMS,11,False,3,null,True,2021-08-26 05:00:00,2.0,normal,NaN
3,1,normal,VURIXMS,11,False,4,null,True,2021-08-26 05:00:00,27.9,normal,NaN
4,3,normal,VURIXMS,11,False,1,c:\,True,2021-08-26 05:00:00,101868.0,normal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
849277,2,normal,VURIX02,102,False,2,drv_huawei,True,2021-08-26 05:24:38,32.3,normal,NaN
849278,2,normal,VURIX02,102,False,2,drv_samsung,True,2021-08-26 05:24:38,144.2,normal,NaN
849279,2,normal,VURIX02,102,False,2,drv_truen,True,2021-08-26 05:24:38,49.7,normal,NaN
849280,2,normal,VURIX02,102,False,2,drv_win4net,True,2021-08-26 05:24:38,15.7,normal,NaN


In [301]:
df_new = df_cc.groupby(['srv_name', 'srv_serial', 'groups_id', 'item_id', 'item_param'], dropna=False).agg(
        recv_count = ('srv_name', 'count'),
        item_min_time = ('item_time', 'min'),
        item_max_time = ('item_time', 'max'),
        item_value = ('item_value', 'mean'),
        item_value_str = ('item_value_str', 'first')
    ).reset_index().round(3)

In [302]:
df_new

,srv_name,srv_serial,groups_id,item_id,item_param,recv_count,item_min_time,item_max_time,item_value,item_value_str
0,VURIX01,101,1,1,null,720,2021-08-26 05:00:08,2021-08-26 05:59:58,11.506,None
1,VURIX01,101,1,2,null,720,2021-08-26 05:00:08,2021-08-26 05:59:58,14.994,None
2,VURIX01,101,1,3,null,720,2021-08-26 05:00:08,2021-08-26 05:59:58,172.599,None
3,VURIX01,101,1,4,null,720,2021-08-26 05:00:08,2021-08-26 05:59:58,26.126,None
4,VURIX01,101,2,1,drv_dahua,720,2021-08-26 05:00:03,2021-08-26 05:59:53,0.000,None
...,...,...,...,...,...,...,...,...,...,...
1250,VURIXMS,11,4,3,null,722,2021-08-26 05:00:00,2021-08-26 05:59:59,1.000,None
1251,VURIXMS,11,4,5,null,722,2021-08-26 05:00:00,2021-08-26 05:59:59,2.000,None
1252,VURIXMS,11,4,8,null,722,2021-08-26 05:00:00,2021-08-26 05:59:59,NaN,1.6.11.210805
1253,VURIXMS,11,4,9,null,722,2021-08-26 05:00:00,2021-08-26 05:59:59,NaN,OpenSSL 1.0.2k 26 Jan 2017


In [303]:
pk_list = ['srv_name', 'srv_serial', 'groups_id', 'item_id', 'item_param']
df_servers = pd.json_normalize(df['data.param.servers'].explode('data.param.servers')).explode('groups').reset_index()
df_groups = pd.json_normalize(df_servers['groups']).drop(['status'], axis=1)
df_items = pd.concat([df_servers, df_groups], axis=1).explode('items').drop(['groups'], axis=1).reset_index()
df_ = pd.concat([df_items, pd.json_normalize(df_items['items'])], axis=1)

df_['item_time'] = pd.to_datetime(df_['item_time']/10000000 - 11644473600, unit='s').astype('datetime64[s]')
df_['item_param'] = df_['item_param'].apply(lambda x : 'null' if 'null' in x else list(json.loads(x).values())[0])
df_['item_value_str'] = df_['item_value'].apply(lambda x : np.nan if x.replace('.','').isdecimal() and x.count('.') <= 1 else x)
df_['item_value']= df_['item_value'].apply(lambda x : pd.to_numeric(x) if x.replace('.','').isdecimal() and x.count('.') <= 1 else np.nan)
df_['groups_id'] = df_['group_id']

df_new = df_.groupby(pk_list, dropna=False).agg(
    recv_count = ('srv_name', 'count'),
    item_min_time = ('item_time', 'min'),
    item_max_time = ('item_time', 'max'),
    item_value = ('item_value', 'mean'),
    item_value_str = ('item_value_str', 'first')
).reset_index().round(3)

In [304]:
df_

,level_0,index,app_type,srv_addr,srv_name,srv_serial,status,type,vms_id,group_id,items,item_changed,item_id,item_param,item_save,item_time,item_value,status,item_value_str,groups_id
0,0,0,0,localhost,VURIXMS,11,normal,state,100381,1,"{'item_changed': False, 'item_id': 1, 'item_pa...",False,1,null,True,2021-08-26 05:00:00,5.0,normal,NaN,1
1,0,0,0,localhost,VURIXMS,11,normal,state,100381,1,"{'item_changed': False, 'item_id': 2, 'item_pa...",False,2,null,True,2021-08-26 05:00:00,32.0,normal,NaN,1
2,0,0,0,localhost,VURIXMS,11,normal,state,100381,1,"{'item_changed': False, 'item_id': 3, 'item_pa...",False,3,null,True,2021-08-26 05:00:00,2.0,normal,NaN,1
3,0,0,0,localhost,VURIXMS,11,normal,state,100381,1,"{'item_changed': False, 'item_id': 4, 'item_pa...",False,4,null,True,2021-08-26 05:00:00,27.9,normal,NaN,1
4,1,0,0,localhost,VURIXMS,11,normal,state,100381,3,"{'item_changed': False, 'item_id': 1, 'item_pa...",False,1,c:\,True,2021-08-26 05:00:00,101868.0,normal,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849277,145229,24555,1,172.23.15.142,VURIX02,102,normal,state,100381,2,"{'item_changed': False, 'item_id': 2, 'item_pa...",False,2,drv_huawei,True,2021-08-26 05:24:38,32.3,normal,NaN,2
849278,145229,24555,1,172.23.15.142,VURIX02,102,normal,state,100381,2,"{'item_changed': False, 'item_id': 2, 'item_pa...",False,2,drv_samsung,True,2021-08-26 05:24:38,144.2,normal,NaN,2
849279,145229,24555,1,172.23.15.142,VURIX02,102,normal,state,100381,2,"{'item_changed': False, 'item_id': 2, 'item_pa...",False,2,drv_truen,True,2021-08-26 05:24:38,49.7,normal,NaN,2
849280,145229,24555,1,172.23.15.142,VURIX02,102,normal,state,100381,2,"{'item_changed': False, 'item_id': 2, 'item_pa...",False,2,drv_win4net,True,2021-08-26 05:24:38,15.7,normal,NaN,2


In [325]:
df_new = df_.groupby(pk_list, dropna=False).agg(
    recv_count = ('srv_name', 'count'),
    item_min_time = ('item_time', 'min'),
    item_max_time = ('item_time', 'max'),
    item_value = ('item_value', 'mean'),
    item_value_str = ('item_value_str', 'first')
).reset_index().round(3)
mask1 = (df_new['recv_count'] <= 360)
df_new.loc[mask1,:]

,srv_name,srv_serial,groups_id,item_id,item_param,recv_count,item_min_time,item_max_time,item_value,item_value_str
37,VURIX02,102,1,1,null,292,2021-08-26 05:00:05,2021-08-26 05:24:28,1.577400e+01,None
38,VURIX02,102,1,2,null,292,2021-08-26 05:00:05,2021-08-26 05:24:28,1.463700e+01,None
39,VURIX02,102,1,3,null,292,2021-08-26 05:00:05,2021-08-26 05:24:28,1.532970e+02,None
40,VURIX02,102,1,4,null,292,2021-08-26 05:00:05,2021-08-26 05:24:28,1.106300e+01,None
41,VURIX02,102,2,1,drv_arecontvision,294,2021-08-26 05:00:00,2021-08-26 05:24:38,0.000000e+00,None
42,VURIX02,102,2,1,drv_commonrtsp,294,2021-08-26 05:00:00,2021-08-26 05:24:38,0.000000e+00,None
43,VURIX02,102,2,1,drv_doowon,294,2021-08-26 05:00:00,2021-08-26 05:24:38,0.000000e+00,None
44,VURIX02,102,2,1,drv_hikvision,294,2021-08-26 05:00:00,2021-08-26 05:24:38,0.000000e+00,None
45,VURIX02,102,2,1,drv_huawei,294,2021-08-26 05:00:00,2021-08-26 05:24:38,0.000000e+00,None
46,VURIX02,102,2,1,drv_samsung,294,2021-08-26 05:00:00,2021-08-26 05:24:38,0.000000e+00,None


In [306]:
import psutil

In [307]:
# 사용중인 CPU 사용량
psutil.cpu_percent()

20.0

In [308]:
# 사용중인 메모리 리소스 정보
psutil.virtual_memory()

svmem(total=34195357696, available=6791319552, percent=80.1, used=27404038144, free=6791319552)

In [309]:
# 사용중인 리소스 딕셔너리 타입 
dict(psutil.virtual_memory()._asdict())

{'total': 34195357696,
 'available': 6791319552,
 'percent': 80.1,
 'used': 27404038144,
 'free': 6791319552}

In [310]:
# 사용중인 리소스 딕셔너리 타입 
dict(psutil.virtual_memory()._asdict())['used']/1024/1024/1024

25.52224349975586

In [311]:
# 사용중인 메모리 사용량 %
psutil.virtual_memory().percent

80.1

In [312]:
# 사용중인 메모리 사용 가능 %
psutil.virtual_memory().available * 100 / psutil.virtual_memory().total

19.8687401149623